# Actividad Cloud Computing 

## 1. Mostrar base de datos en SQL
Abran un servidor en Azure con la base de datos de prueba de SQL. 

Usamos ODBC para jalar los datos a Python

Asegurando su instalación:

In [110]:
import sys
import subprocess

# Install pyodbc
subprocess.check_call([sys.executable, "-m", "pip", "install", "sqlalchemy"])

0

In [111]:
import pyodbc # type: ignore
import json

# Load the connection details from the JSON file
with open("connection_details.json", "r") as json_file:
    connection_details = json.load(json_file)

# Extract the connection details
server = connection_details["server"]
database = connection_details["database"]
username = connection_details["username"]
password = connection_details["password"]

# Define the connection string for SQL Server (Azure)
conn_str = (
    f'DRIVER={{ODBC Driver 17 for SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password}'
)

# Establish the connection
try:
    conn = pyodbc.connect(conn_str)
    print("Connected to the Azure SQL Database successfully!")

except pyodbc.Error as e:
    print(f"Error connecting to the database: {e}")


Connected to the Azure SQL Database successfully!


Se logra establecer conexión con la base de datos de prueba de SQL. 

## 2. Entrenar modelo usando dicha base de datos

El modelo, una regresión, deberá predecir fecha de modificación de la tabla SalesLT.costumer 

Se tienen los datos de SQL en `datos` en python

In [112]:
import pandas as pd 
query = "SELECT * FROM SalesLT.customer"
datos = pd.read_sql(query, conn)
datos.head()

C:\Users\David\AppData\Local\Temp\ipykernel_32524\1185087627.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  datos = pd.read_sql(query, conn)


,CustomerID,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,CompanyName,SalesPerson,EmailAddress,Phone,PasswordHash,PasswordSalt,rowguid,ModifiedDate
0,1,False,Mr.,Orlando,N.,Gee,None,A Bike Store,adventure-works\pamela0,orlando0@adventure-works.com,245-555-0173,L/Rlwxzp4w7RWmEgXX+/A7cXaePEPcp+KwQhl2fJL7w=,1KjXYs4=,3F5AE95E-B87D-4AED-95B4-C3797AFCB74F,2005-08-01
1,2,False,Mr.,Keith,None,Harris,None,Progressive Sports,adventure-works\david8,keith0@adventure-works.com,170-555-0127,YPdtRdvqeAhj6wyxEsFdshBDNXxkCXn+CRgbvJItknw=,fs1ZGhY=,E552F657-A9AF-4A7D-A645-C429D6E02491,2006-08-01
2,3,False,Ms.,Donna,F.,Carreras,None,Advanced Bike Components,adventure-works\jillian0,donna0@adventure-works.com,279-555-0130,LNoK27abGQo48gGue3EBV/UrlYSToV0/s87dCRV7uJk=,YTNH5Rw=,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2005-09-01
3,4,False,Ms.,Janet,M.,Gates,None,Modular Cycle Systems,adventure-works\jillian0,janet1@adventure-works.com,710-555-0173,ElzTpSNbUW1Ut+L5cWlfR7MF6nBZia8WpmGaQPjLOJA=,nm7D5e4=,FF862851-1DAA-4044-BE7C-3E85583C054D,2006-07-01
4,5,False,Mr.,Lucy,None,Harrington,None,Metropolitan Sports Supply,adventure-works\shu0,lucy0@adventure-works.com,828-555-0186,KJqV15wsX3PG8TS5GSddp6LFFVdd3CoRftZM/tP0+R4=,cNFKU4w=,83905BDC-6F5E-4F71-B162-C98DA069F38A,2006-09-01


In [113]:
datos.columns

Index(['CustomerID', 'NameStyle', 'Title', 'FirstName', 'MiddleName',
       'LastName', 'Suffix', 'CompanyName', 'SalesPerson', 'EmailAddress',
       'Phone', 'PasswordHash', 'PasswordSalt', 'rowguid', 'ModifiedDate'],
      dtype='object')

Asegurando que tenemos los datos, podemos cerrar la conexión. 

In [114]:
conn.close()

Se predecirá `ModifiedDate` según otros datos numéricos en la base de datos, como lo son `CustomerID` y `Phone`

In [115]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [116]:
datos = datos[['CustomerID', 'Phone', 'ModifiedDate']]

In [ ]:
datos['Phone'] = datos['Phone'].str.replace('-', '')
datos['Phone'] = datos['Phone'].str.replace(' ', '')
datos['Phone'] = datos['Phone'].str.replace('(', '')
datos['Phone'] = datos['Phone'].str.replace(')', '')
datos['Phone'] = datos['Phone'].str.replace('+', '')
datos['Phone'] = datos['Phone'].astype(np.float64)
datos

,CustomerID,Phone,ModifiedDate
0,1,2.455550e+09,2005-08-01
1,2,1.705550e+09,2006-08-01
2,3,2.795550e+09,2005-09-01
3,4,7.105550e+09,2006-07-01
4,5,8.285550e+09,2006-09-01
...,...,...,...
842,30113,1.115006e+12,2006-09-01
843,30115,1.555550e+09,2006-08-01
844,30116,4.335550e+09,2007-07-01
845,30117,5.605550e+09,2005-08-01


In [118]:
datos['ModifiedDate'] = pd.to_datetime(datos['ModifiedDate']) 
datos['ModifiedDate'] = datos['ModifiedDate'].map(pd.Timestamp.timestamp)
datos

,CustomerID,Phone,ModifiedDate
0,1,2.455550e+09,1.122854e+09
1,2,1.705550e+09,1.154390e+09
2,3,2.795550e+09,1.125533e+09
3,4,7.105550e+09,1.151712e+09
4,5,8.285550e+09,1.157069e+09
...,...,...,...
842,30113,1.115006e+12,1.157069e+09
843,30115,1.555550e+09,1.154390e+09
844,30116,4.335550e+09,1.183248e+09
845,30117,5.605550e+09,1.122854e+09


In [119]:
numbers = ['CustomerID', 'Phone']
X = datos[numbers]
X = X.fillna(0)  
y = datos['ModifiedDate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [120]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [121]:
mse = mean_squared_error(y_test, y_pred)
print(f"Error cuadrado medio: {mse}")
r2 = r2_score(y_test, y_pred)
print(f"R^2: {r2}")

Error cuadrado medio: 809698940300736.9
R^2: 0.017466341889975534


Para interpretar estos números, es conveniente convertir los números de regreso a valores de fechas

In [122]:
y_test_dates = pd.to_datetime(y_test, unit = 's') # Valores originales
y_pred_dates = pd.to_datetime(y_pred, unit = 's') # Valores predichos 

Y se le da una interpretación a los errores obtenidos en días. Lo siguiente fue hecho con ayuda de IA. 

In [123]:
# Absolute error in days
absolute_errors = (y_test_dates - y_pred_dates).abs()
absolute_errors_in_days = absolute_errors.dt.total_seconds() / (60 * 60 * 24)  # Convert seconds to days

# Mean Absolute Error (in days)
mae_days = absolute_errors_in_days.mean()
print(f"Mean Absolute Error (MAE) in days: {mae_days:.2f}")

# Evaluate Mean Squared Error (MSE) and R² for reference
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R²): {r2}")

Mean Absolute Error (MAE) in days: 270.34
Mean Squared Error (MSE): 809698940300736.9
R-squared (R²): 0.017466341889975534


Podemos mostrar algunos ejemplos de las fechas originales vs. las fechas predichas para observar los errores del modelo. 

In [124]:
compara = pd.DataFrame({
    'Fecha real': y_test_dates,
    'Fecha predicha': y_pred_dates.round('s'),
    'Error absoluto (días)': absolute_errors_in_days.round(2)
})
print(compara.head())

    Fecha real      Fecha predicha  Error absoluto (días)
434 2006-08-01 2006-07-03 04:28:38                  28.81
780 2006-09-01 2007-02-11 22:17:35                 163.93
620 2006-07-01 2007-02-12 02:40:07                 226.11
545 2006-07-01 2006-06-10 11:00:52                  20.54
484 2005-09-01 2006-06-10 09:16:30                 282.39


Ahora exportamos el modelo como archivo .pkl

In [125]:
import joblib

joblib.dump(model, 'model_regression.pkl')
print("Modelo exportado como 'model_regression.pkl'")

Modelo exportado como 'model_regression.pkl'
